In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
import pathlib

import cdmetadl

PROJECT_DIR = pathlib.Path(cdmetadl.__file__).parent.parent

In [34]:
import cdmetadl.samplers
import cdmetadl.helpers.general_helpers
import cdmetadl.dataset

path_to_dataset = "/fastdata/vilab24/meta-album"
dataset_path = pathlib.Path(path_to_dataset)
dataset_info_dict = cdmetadl.helpers.general_helpers.check_datasets(dataset_path, ["BTS"])
dataset = cdmetadl.dataset.ImageDataset("Boats", dataset_info_dict["BTS"])

n_way_sampler = cdmetadl.samplers.ValueSampler(value=5)
k_shot_sampler = cdmetadl.samplers.ValueSampler(value=21)
task = dataset.generate_task(n_way_sampler, k_shot_sampler, query_size=10)

In [43]:
import cdmetadl.helpers.general_helpers
import cdmetadl.api
import cdmetadl.confidence
import cdmetadl.augmentation
import cdmetadl.helpers.scoring_helpers

model_name = "finetuning"
model_dir = f"/fastdata/vilab24/output/final/training/dropout_30_k_1/{model_name}/cross-domain/model"

device = cdmetadl.helpers.general_helpers.get_device()

model_module = cdmetadl.helpers.general_helpers.load_module_from_path(PROJECT_DIR / f"./baselines/{model_name}/model.py")
confidence_learner: cdmetadl.api.Learner = model_module.MyLearner()
learner: cdmetadl.api.Learner = model_module.MyLearner()

confidence_estimator = cdmetadl.confidence.PseudoConfidenceEstimator()
augmentor = cdmetadl.augmentation.PseudoAugmentation(augmentation_size={
    "scale": 1,
    "offset": 3 ,
    "maximum": 20,
    "threshold": 0.8,
},
keep_original_data=True, device=device)

task.support_set.images = task.support_set.images.to(device)
task.support_set.labels = task.support_set.labels.to(device)
task.query_set.images = task.query_set.images.to(device)
task.query_set.labels = task.query_set.labels.to(device)

original_number_of_shots = task.support_set.min_number_of_shots

confidence_learner.load(model_dir)
# Adjust T for finetuning
confidence_learner.T = 1000
support_set, confidence_scores = confidence_estimator.estimate(confidence_learner, task.support_set)

support_set_without_confidence = support_set

learner.load(model_dir)
# Adjust T for finetuning
learner.T = 1000
predictor = learner.fit(task.support_set)

pred_dict_prev = {
    "Dataset": task.dataset_name,
    "Number of Ways": task.number_of_ways,
    "Number of Shots": original_number_of_shots,
    "Number of Shots per Class": support_set.number_of_shots_per_class,
    "Confidence Scores": confidence_scores,
    "Predictions": predictor.predict(task.query_set.images).cpu().numpy(),
    "Ground Truth": task.query_set.labels.cpu().numpy(),
}

scores_prev = cdmetadl.helpers.scoring_helpers.compute_all_scores(
    pred_dict_prev["Ground Truth"], pred_dict_prev["Predictions"], pred_dict_prev["Number of Ways"]
)

if augmentor is not None:
    # print(confidence_scores)
    support_set = augmentor.augment(support_set, conf_scores=confidence_scores)

learner.load(model_dir)
# Adjust T for finetuning
learner.T = 1000
predictor = learner.fit(support_set)

pred_dict = {
    "Dataset": task.dataset_name,
    "Number of Ways": task.number_of_ways,
    "Number of Shots": original_number_of_shots,
    "Number of Shots per Class": support_set.number_of_shots_per_class,
    "Confidence Scores": confidence_scores,
    "Predictions": predictor.predict(task.query_set.images).cpu().numpy(),
    "Ground Truth": task.query_set.labels.cpu().numpy(),
}

scores = cdmetadl.helpers.scoring_helpers.compute_all_scores(
    pred_dict["Ground Truth"], pred_dict["Predictions"], pred_dict["Number of Ways"]
)

In [57]:
import numpy as np
from collections import defaultdict

confidence_learner.load(model_dir)
# Adjust T for finetuning
confidence_learner.T = 1000

predictor = confidence_learner.fit(support_set_without_confidence)

confidence_scores = defaultdict(list)
for prediction, gt_label in zip(predictor.predict(task.query_set.images).cpu().numpy(), task.query_set.labels):
    gt_label = int(gt_label)

    confidence_score = 0.0
    if np.argmax(prediction) == gt_label:
        confidence_score = np.max(prediction)

    confidence_scores[gt_label].append(confidence_score)

[np.mean(scores) for scores in confidence_scores.values()]

# [0.06470633745193481, 0.08580020368099213, 0.5084564238786697, 0.1895177721977234, 0.032207930088043214]
# [0.61396587, 0.5356838136911393, 0.87390435, 0.74509877, 0.6357733845710755]

[0.04358408153057099,
 0.21569089889526366,
 0.1704721122980118,
 0.22646460235118865,
 0.08967948853969573]

In [51]:
import pandas as pd

print(pd.Series(scores_prev))
print(pred_dict["Confidence Scores"])
print(pred_dict["Number of Shots per Class"])
print(pd.Series(scores))

Normalized Accuracy    0.175000
Accuracy               0.340000
Macro F1 Score         0.288329
Macro Precision        0.269398
Macro Recall           0.340000
dtype: float64
[0.06470633745193481, 0.08580020368099213, 0.5084564238786697, 0.1895177721977234, 0.032207930088043214]
[5 5 2 4 5]
Normalized Accuracy    0.175000
Accuracy               0.340000
Macro F1 Score         0.352974
Macro Precision        0.393086
Macro Recall           0.340000
dtype: float64


In [52]:
import cdmetadl.helpers.general_helpers
import cdmetadl.api
import cdmetadl.confidence
import cdmetadl.augmentation
import cdmetadl.helpers.scoring_helpers

model_name = "finetuning"
model_dir = f"/fastdata/vilab24/output/final/training/dropout_30_k_1/{model_name}/cross-domain/model"

device = cdmetadl.helpers.general_helpers.get_device()

model_module = cdmetadl.helpers.general_helpers.load_module_from_path(
    PROJECT_DIR / f"./baselines/{model_name}/model.py"
)
confidence_learner: cdmetadl.api.Learner = model_module.MyLearner()
learner: cdmetadl.api.Learner = model_module.MyLearner()

confidence_estimator = cdmetadl.confidence.ConstantConfidenceProvider(confidence=0.20)
augmentor = cdmetadl.augmentation.PseudoAugmentation(
    augmentation_size={
        "scale": 1,
        "offset": 3,
        "maximum": 20,
        "threshold": 0.8,
    }, keep_original_data=True, device=device
)

task.support_set.images = task.support_set.images.to(device)
task.support_set.labels = task.support_set.labels.to(device)
task.query_set.images = task.query_set.images.to(device)
task.query_set.labels = task.query_set.labels.to(device)

original_number_of_shots = task.support_set.min_number_of_shots

confidence_learner.load(model_dir)
# Adjust T for finetuning
confidence_learner.T = 1000
support_set, confidence_scores = confidence_estimator.estimate(confidence_learner, support_set_without_confidence)

learner.load(model_dir)
# Adjust T for finetuning
learner.T = 1000
predictor = learner.fit(task.support_set)

pred_dict_prev_all = {
    "Dataset": task.dataset_name,
    "Number of Ways": task.number_of_ways,
    "Number of Shots": original_number_of_shots,
    "Number of Shots per Class": support_set.number_of_shots_per_class,
    "Confidence Scores": confidence_scores,
    "Predictions": predictor.predict(task.query_set.images).cpu().numpy(),
    "Ground Truth": task.query_set.labels.cpu().numpy(),
}

scores_prev_all = cdmetadl.helpers.scoring_helpers.compute_all_scores(
    pred_dict_prev_all["Ground Truth"], pred_dict_prev_all["Predictions"], pred_dict_prev_all["Number of Ways"]
)

if augmentor is not None:
    # print(confidence_scores)
    support_set = augmentor.augment(support_set, conf_scores=confidence_scores)

learner.load(model_dir)
# Adjust T for finetuning
learner.T = 1000
predictor = learner.fit(support_set)

pred_dict_all = {
    "Dataset": task.dataset_name,
    "Number of Ways": task.number_of_ways,
    "Number of Shots": original_number_of_shots,
    "Number of Shots per Class": support_set.number_of_shots_per_class,
    "Confidence Scores": confidence_scores,
    "Predictions": predictor.predict(task.query_set.images).cpu().numpy(),
    "Ground Truth": task.query_set.labels.cpu().numpy(),
}

scores_all = cdmetadl.helpers.scoring_helpers.compute_all_scores(
    pred_dict_all["Ground Truth"], pred_dict_all["Predictions"], pred_dict_all["Number of Ways"]
)

In [54]:
print(pd.Series(scores_prev_all))
print(pred_dict_all["Confidence Scores"])
print(pred_dict_all["Number of Shots per Class"])
print(pd.Series(scores_all))

Normalized Accuracy    0.175000
Accuracy               0.340000
Macro F1 Score         0.320000
Macro Precision        0.347255
Macro Recall           0.340000
dtype: float64
[0.2, 0.2, 0.2, 0.2, 0.2]
[4 4 4 4 4]
Normalized Accuracy    0.275000
Accuracy               0.420000
Macro F1 Score         0.399434
Macro Precision        0.456224
Macro Recall           0.420000
dtype: float64


In [60]:
import cdmetadl.helpers.general_helpers
import cdmetadl.api
import cdmetadl.confidence
import cdmetadl.augmentation
import cdmetadl.helpers.scoring_helpers

model_name = "finetuning"
model_dir = f"/fastdata/vilab24/output/final/training/dropout_30_k_1/{model_name}/cross-domain/model"

device = cdmetadl.helpers.general_helpers.get_device()

model_module = cdmetadl.helpers.general_helpers.load_module_from_path(
    PROJECT_DIR / f"./baselines/{model_name}/model.py"
)
confidence_learner: cdmetadl.api.Learner = model_module.MyLearner()
learner: cdmetadl.api.Learner = model_module.MyLearner()

confidence_estimator = cdmetadl.confidence.ConstantConfidenceProvider(confidence=0.20)
augmentor = cdmetadl.augmentation.PseudoAugmentation(
    augmentation_size={
        "scale": 1,
        "offset": 3,
        "maximum": 20,
        "threshold": 0.8,
    }, keep_original_data=True, device=device
)

task.support_set.images = task.support_set.images.to(device)
task.support_set.labels = task.support_set.labels.to(device)
task.query_set.images = task.query_set.images.to(device)
task.query_set.labels = task.query_set.labels.to(device)

original_number_of_shots = task.support_set.min_number_of_shots

confidence_learner.load(model_dir)
# Adjust T for finetuning
confidence_learner.T = 1000
support_set = support_set_without_confidence


learner.load(model_dir)
# Adjust T for finetuning
learner.T = 1000
predictor = learner.fit(task.support_set)

predictions = predictor.predict(task.query_set.images).cpu().numpy()

confidence_scores = defaultdict(list)
for prediction, gt_label in zip(predictions, task.query_set.labels):
    gt_label = int(gt_label)

    confidence_score = 0.0
    if np.argmax(prediction) == gt_label:
        confidence_score = np.max(prediction)

    confidence_scores[gt_label].append(confidence_score)

confidence_scores = [np.mean(scores) for scores in confidence_scores.values()]

pred_dict_prev_all = {
    "Dataset": task.dataset_name,
    "Number of Ways": task.number_of_ways,
    "Number of Shots": original_number_of_shots,
    "Number of Shots per Class": support_set.number_of_shots_per_class,
    "Confidence Scores": confidence_scores,
    "Predictions": predictions,
    "Ground Truth": task.query_set.labels.cpu().numpy(),
}


scores_prev_all = cdmetadl.helpers.scoring_helpers.compute_all_scores(
    pred_dict_prev_all["Ground Truth"], pred_dict_prev_all["Predictions"], pred_dict_prev_all["Number of Ways"]
)

if augmentor is not None:
    # print(confidence_scores)
    support_set = augmentor.augment(support_set, conf_scores=confidence_scores)

learner.load(model_dir)
# Adjust T for finetuning
learner.T = 1000
predictor = learner.fit(support_set)

pred_dict_all = {
    "Dataset": task.dataset_name,
    "Number of Ways": task.number_of_ways,
    "Number of Shots": original_number_of_shots,
    "Number of Shots per Class": support_set.number_of_shots_per_class,
    "Confidence Scores": confidence_scores,
    "Predictions": predictor.predict(task.query_set.images).cpu().numpy(),
    "Ground Truth": task.query_set.labels.cpu().numpy(),
}

scores_all = cdmetadl.helpers.scoring_helpers.compute_all_scores(
    pred_dict_all["Ground Truth"], pred_dict_all["Predictions"], pred_dict_all["Number of Ways"]
)

In [61]:
print(pd.Series(scores_prev_all))
print(pred_dict_all["Confidence Scores"])
print(pred_dict_all["Number of Shots per Class"])
print(pd.Series(scores_all))

Normalized Accuracy    0.175000
Accuracy               0.340000
Macro F1 Score         0.320000
Macro Precision        0.347255
Macro Recall           0.340000
dtype: float64
[0.13608461618423462, 0.31828450262546537, 0.23118655383586884, 0.15830894708633422, 0.02658001482486725]
[4 3 4 4 5]
Normalized Accuracy    0.250000
Accuracy               0.400000
Macro F1 Score         0.392721
Macro Precision        0.492794
Macro Recall           0.400000
dtype: float64
